In [17]:
# Will need to run the following in anaconda before
# pip install pandas-datareader
# pip install PyPortfolioOpt


SyntaxError: invalid syntax (<ipython-input-17-2e32e3c3c9e6>, line 2)

In [ ]:
# Import the necessary packages
import pandas-datareader.data as pdata
import datetime
import pandas as pd
from functools import reduce
from pypfopt.expected_returns import mean_historial_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import HRPOpt

In [ ]:
# Limit the display limits on the rows and the columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Define the period we want to look over
start = datetime.datetime(2022, 1, 1)
end = datetime.datetime(2023, 12, 31)

In [ ]:
# Create a function that will pull the data for several stocks
def get_stock(ticker):
    data = pdata.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Close"]
    data = data[[f'{ticker}']]
    print(data.head())
    return data
         

In [ ]:
# Create another function that takes a list of stocks and generates a single data frame of stock prices for each stock
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
        
    df_merged = reduce(lambda, left,right: pd.merge(left, right, on=['Date'], how='outer'), date_frames)
    print(df_merged.head())
    return df_merged
# Create a portfolio from the following stocks: Apple, Astrazeneca, JP Morgan, Accenture, Coca Cola, Boeing, Walmart, Shell, Vanguard, GSK
stocks = ["AAPL", "AZN.L", "JPM", "ACN", "CCH.L", "BA", "WMT", "SHEL.L", "VTI", "GSK.L"]
portfolio = combine_stocks(stocks)

In [ ]:
# Save the prices as a csv file
portfolio.to_csv("portfolio.csv", index=False)
portfolio = pd.read_csv("portfolio.csv")

In [ ]:
# Markowtiz optimisation
# Find the mean historical return and covariance matrix
mu = mean_histotical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()

In [ ]:
# Determine the Efficient Frontier and weights 
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(dict(cleaned_weights))

In [ ]:
# Display portfolio performance
ef.portfolio_performance(verbose=True)

In [ ]:
# Convert Weights into allocations from an investment of $100,000
latest_prices = get_latest_prices(portfolio)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)
allocation, leftover = da.greedy_portfolio
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

In [ ]:
# Hierachicial Risk Parity
# Determine returns
returns = portfolio.pct_change().dropna()

In [ ]:
# Run optimisaiton algorithm to get the weights
hrp = HRPOpt(returns)
hrp_weights = hrp.optimize()

In [ ]:
# Print performance of portfolio and weights
hrp.portfolio_performance(verbose=True)
print(dict(hrp_weights))

In [ ]:
# Determine discrete allocation using weights
da_hrp = DiscreteAllocation(hrp_weights, latest_prices, total_portfolio_value=100000)
allocation, leftover = da_hrp.greedy_portfolio()
print("Discrete allocation (HRP):", allocation)
print("Funds remaining (HRP): ${:.2f}".format(leftover))

In [ ]:
# Mean conditional value at risk
from pypfopt.efficient_frontier import EfficientCVaR

In [ ]:
# Calculate weights and get performance
S = portfolio.cov()
ef_cvar = EfficientCVaR(mu, S)
cvar_weights = ef_cvar.min_cvar()
cleaned_weights = ef_cvar.clean_weights()
print(dict(cleaned_weights))

In [ ]:
# Get the discrete allocation
da_cvar = DiscreteAllocatoin(cvar_weights, latest_prices, total_portfolio_value = 100000)
allocation, leftover = da_cvar.greedy_portfolio()
print("Discrete allocation (CVAR):", allocation)
print("Funds remaining (CVAR): ${:.2f}".format(leftover))